In [1]:
from sklearn.linear_model import LogisticRegression
import argparse
import os
import numpy as np
from sklearn.metrics import mean_squared_error
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from azureml.core.run import Run
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Workspace, Dataset
from azureml.core import Workspace, Experiment, Run

ws = Workspace(subscription_id="502bce8d-d66b-4661-a423-f21e61d20123",
                  resource_group="aml-quickstarts-127635",
                  workspace_name="quick-starts-ws-127635")
exp = Experiment(workspace=ws, name="udacity-project-hp")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-127635
Azure region: southcentralus
Subscription id: 502bce8d-d66b-4661-a423-f21e61d20123
Resource group: aml-quickstarts-127635


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.


   # Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

   # Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                              vm_priority='low_priority',max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.dnn import TensorFlow
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, normal, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
        '--C': choice(0,0.25,0.5,1)
        '--max_iter': choice(10,50,500,1000)
    }
)

# Specify a Policy
early_termination_policy = BanditPolicy(slack_factor = 0.15, evaluation_interval=2)

if "training" not in os.listdir():
    os.mkdir("./training")
    
script_params={
    '--datastore-dir': ws.get_default_datastore().as_mount(),
}
# Create a SKLearn estimator for use with train.py
sk_estimator = SKLearn(source_directory='training', 
                     script_params=script_params,
                    compute_target=cpu_cluster,
                    entry_script='train.py',
                    pip_packages=['joblib']
                   )

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_run_config = HyperDriveConfig(estimator = sk_estimator, 
                                            hyperparameter_sampling = ps, 
                                            policy = early_termination_policy,
                                            primary_metric_name = "Accuracy",
                                            primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                            max_total_runs = 20,
                                            max_concurrent_runs = 4)

WARNING - You have specified to install packages in your run. Note that you have overridden Azure ML's installation of the following packages: ['joblib']. We cannot guarantee image build will succeed.


In [4]:

# Submit your hyperdrive run to the experiment and show run details with the widget.

hd_run = exp.submit(hyperdrive_run_config)
RunDetails(Run(exp, hd_run.id)).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.
WARNING - If 'arguments' has been provided here and arguments have been specified in 'run_config', 'arguments' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [55]:
import joblib
# Get your best run and save the model from that run.

best_run = hd_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
print('\n learning rate:',parameter_values[3])
print('\n keep probability:',parameter_values[5])
print('\n batch size:',parameter_values[7])

AttributeError: 'NoneType' object has no attribute 'get_metrics'

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
from azureml.core import Dataset

blob_ds = ws.get_default_datastore()
csv_paths = [(blob_ds, 'data/files/current_data.csv')]
ds = Dataset.Tabular.from_delimited_files(path=csv_paths)
ds = ds.register(workspace=ws, name='csv_table')

In [ ]:
from train import clean_data

 
x, y = clean_data(ds)


In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=,
    label_column_name="y",
    n_cross_validations=5)

In [2]:
# Submit your automl run
experiment_name = 'automl-classification'
experiment = Experiment(ws, experiment_name)
run = experiment.submit(automl_config, show_output=True)

In [ ]:
# Retrieve and save your best automl model.
best_run, fitted_model = run.get_output()
print(best_run)
print(fitted_model)

In [ ]:
cpu_cluster.delete()